<a href="https://colab.research.google.com/github/dzejkopjarosinski/ZespolR/blob/main/MLF_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yahoofinance
!pip install yahoofinancials

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 604.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for yahoofinancials: filename=yahoofinancials-1.14-py3-none-any.whl size=28643 sha256=bf6ee56730be73ff6568ff6631b45d6f36b658e0365b58e10d848f183e3b3517
  Stored in directory: /root/.cache/pip/wheels/df/33/51/592213372072da189a4ff8f471838cadcc91add5821f7b9b38
Successfully built yahoofinancials


In [2]:
import pandas as pd
from yahoofinance import BalanceSheet
from yahoofinancials import YahooFinancials
from yahoofinance import IncomeStatement
import pprint

In [3]:
start_date = '2018-01-01'
end_date = '2022-01-01'

In [4]:
tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'GOOGL', 'FB', 'TSLA', 'BRK-A', 'JPM', 'JNJ', 'V', 'PG', 'NVDA', 'MA', 'UNH', 'HD', 'BAC', 'PYPL', 'DIS', 'XOM']

financials = {}
income_statements={}
for ticker in tickers:
    try:
        yahoo_financials = YahooFinancials(ticker)
        balance_sheet_data = yahoo_financials.get_financial_stmts('annual', 'balance')
        income_statement_data = IncomeStatement(ticker)
        income_statements[ticker] = income_statement_data
        financials[ticker] = balance_sheet_data['balanceSheetHistory'][ticker]
    except Exception as e:
        print(f"Failed to download {ticker}: {e}")



In [5]:
financials

{'AAPL': [{'2019-09-30': {'cashAndCashEquivalents': 48844000000.0,
    'stockholdersEquity': 90488000000.0,
    'machineryFurnitureEquipment': 69797000000.0,
    'properties': 0.0,
    'investmentsAndAdvances': 105341000000.0,
    'commonStockEquity': 90488000000.0,
    'availableForSaleSecurities': 105341000000.0,
    'landAndImprovements': 17085000000.0,
    'investmentinFinancialAssets': 105341000000.0,
    'currentDeferredRevenue': 5522000000.0,
    'totalCapitalization': 182295000000.0,
    'capitalStock': 45174000000.0,
    'otherInvestments': 105341000000.0,
    'accountsReceivable': 22926000000.0,
    'shareIssued': 17772944000.0,
    'payablesAndAccruedExpenses': 46236000000.0,
    'totalNonCurrentLiabilitiesNetMinorityInterest': 142310000000.0,
    'otherCurrentBorrowings': 10260000000.0,
    'cashFinancial': 12204000000.0,
    'longTermDebtAndCapitalLeaseObligation': 91807000000.0,
    'otherShortTermInvestments': 51713000000.0,
    'cashCashEquivalentsAndShortTermInvestment

In [27]:
financials_calculated ={}
for ticker in tickers:
  financials_calculated[ticker] = []
  for i,each in enumerate(financials[ticker]):
    datekeys = list(each.keys())
    datekey = datekeys[0] 
    # Profitability ratios
    emptydict={}
    financials_calculated[ticker].append(emptydict)
    financials_calculated[ticker][i][datekey] = {}

    try:
      financials_calculated[ticker][i][datekey]["net_income"] = -1 * each[datekey]['gainsLossesNotAffectingRetainedEarnings'] + each[datekey]['retainedEarnings']
    except Exception as e:
        print(f"Failed to calculate {datekey}: {e}")

    try:
      financials_calculated[ticker][i][datekey]["total_revenue"] = each[datekey]['totalRevenue']
    except Exception as e:
        print(f"Failed to calculate {datekey}: {e}")

    try:
      financials_calculated[ticker][i][datekey]["profit_margin"] = financials_calculated[ticker][i][datekey]["net_income"] / financials_calculated[ticker][i][datekey]["total_revenue"]
    except Exception as e:
        print(f"Failed to calculate {datekey}: {e}")

    # Liquidity ratios
    try:
      financials_calculated[ticker][i][datekey]["current_assets"] = each[datekey]['cashAndCashEquivalents'] + each[datekey]['accountsReceivable'] + each[datekey]['inventory']
    except Exception as e:
        print(f"Failed to calculate {datekey}: {e}")

    try:
      financials_calculated[ticker][i][datekey]["current_liabilities"] = each[datekey]['currentLiabilities']
    except Exception as e:
        print(f"Failed to calculate {datekey}: {e}")

    try:
      financials_calculated[ticker][i][datekey]["current_ratio"] = financials_calculated[ticker][i][datekey]["current_assets"] / financials_calculated[ticker][i][datekey]["current_liabilities"]
    except Exception as e:
        print(f"Failed to calculate {datekey}: {e}")

    # Solvency ratios
    try:
      financials_calculated[ticker][i][datekey]["total_asset"] = each[datekey]['totalAssets']
    except Exception as e:
        print(f"Failed to calculate {datekey}: {e}")

    try:
      financials_calculated[ticker][i][datekey]["total_liabilities"] = each[datekey]['totalLiabilitiesNetMinorityInterest']
    except Exception as e:
        print(f"Failed to calculate {datekey}: {e}")

    try:
      financials_calculated[ticker][i][datekey]["debt_to_equity_ratio"] = financials_calculated[ticker][i][datekey]["total_liabilities"] / each[datekey]['commonStockEquity']
    except Exception as e:
        print(f"Failed to calculate {datekey}: {e}")

    try:
      financials_calculated[ticker][i][datekey]["debt_to_assets_ratio"] = financials_calculated[ticker][i][datekey]["total_liabilities"] / financials_calculated[ticker][i][datekey]["total_asset"]
    except Exception as e:
        print(f"Failed to calculate {datekey}: {e}")


    # Valuation ratios
    try:
      financials_calculated[ticker][i][datekey]["market_cap"] = each[datekey]['marketCap']
    except Exception as e:
      print(f"Failed to calculate {datekey}: {e}")

    try:
      financials_calculated[ticker][i][datekey]["enterprise_value"] = each[datekey]['totalEnterpriseValue']
    except Exception as e:
      print(f"Failed to calculate {datekey}: {e}")

    try:
      financials_calculated[ticker][i][datekey]["ev_to_ebitda"] = financials_calculated[ticker][i][datekey]["enterprise_value"] / each[datekey]['ebitda']
    except Exception as e:
      print(f"Failed to calculate {datekey}: {e}")
        

Failed to calculate 2019-09-30: 'totalRevenue'
Failed to calculate 2019-09-30: 'total_revenue'
Failed to calculate 2019-09-30: 'marketCap'
Failed to calculate 2019-09-30: 'totalEnterpriseValue'
Failed to calculate 2019-09-30: 'enterprise_value'
Failed to calculate 2020-09-30: 'totalRevenue'
Failed to calculate 2020-09-30: 'total_revenue'
Failed to calculate 2020-09-30: 'marketCap'
Failed to calculate 2020-09-30: 'totalEnterpriseValue'
Failed to calculate 2020-09-30: 'enterprise_value'
Failed to calculate 2021-09-30: 'totalRevenue'
Failed to calculate 2021-09-30: 'total_revenue'
Failed to calculate 2021-09-30: 'marketCap'
Failed to calculate 2021-09-30: 'totalEnterpriseValue'
Failed to calculate 2021-09-30: 'enterprise_value'
Failed to calculate 2022-09-30: 'totalRevenue'
Failed to calculate 2022-09-30: 'total_revenue'
Failed to calculate 2022-09-30: 'marketCap'
Failed to calculate 2022-09-30: 'totalEnterpriseValue'
Failed to calculate 2022-09-30: 'enterprise_value'
Failed to calculate 

In [28]:
financials_calculated

{'AAPL': [{'2019-09-30': {'net_income': 46482000000.0,
    'current_assets': 75876000000.0,
    'current_liabilities': 105718000000.0,
    'current_ratio': 0.7177207287311527,
    'total_asset': 338516000000.0,
    'total_liabilities': 248028000000.0,
    'debt_to_equity_ratio': 2.7410043320661304,
    'debt_to_assets_ratio': 0.7326921031797611}},
  {'2020-09-30': {'net_income': 15372000000.0,
    'current_assets': 58197000000.0,
    'current_liabilities': 105392000000.0,
    'current_ratio': 0.5521956125702141,
    'total_asset': 323888000000.0,
    'total_liabilities': 258549000000.0,
    'debt_to_equity_ratio': 3.957039440456695,
    'debt_to_assets_ratio': 0.7982666847799239}},
  {'2021-09-30': {'net_income': 5399000000.0,
    'current_assets': 67798000000.0,
    'current_liabilities': 125481000000.0,
    'current_ratio': 0.5403049067189455,
    'total_asset': 351002000000.0,
    'total_liabilities': 287912000000.0,
    'debt_to_equity_ratio': 4.563512442542399,
    'debt_to_assets

In [ ]:
# Profitability ratios
financials['AAPL']['net_income'] / financials['AAPL']['total_revenue']

# Liquidity ratios
financials['AAPL']['current_assets'] / financials['AAPL']['current_liabilities']

# Solvency ratios
financials['AAPL']['total_assets'] / financials['AAPL']['total_liabilities']

# Valuation ratios
financials['AAPL']['market_cap'] / financials['AAPL']['ebitda']


In [ ]:
rankings = {}
for ticker, data in financials.items():
    try:
        rankings[ticker] = data['net_income'] / data['total_revenue']
    except Exception as e:
        print(f"Failed to calculate ranking for {ticker}: {e}")

rankings = pd.DataFrame(rankings)
rankings = rankings.rank(axis=1, ascending=False)


In [ ]:
scores = rankings.sum()


In [ ]:
top_10 = scores.nlargest(10)
